# AAV5/AAV9 Surface Aromatics

#### Background
We have generated data that suggests AAV5 and AAV9 have differential absorbance/capsid ratios at 260nm and 280nm wavelengths. When denatured, this difference appears minimized. The leading hypothesis is that this is caused by differences in aromatic residues presented on the surface of the capsid for each serotype.

#### Function
This notebook details code to find mismatching aromatic residues in AAV5 and AAV9 capsid sequences. I then use a SASA algorithm from Biopython to estimate which of these residues is on the surface of the capsid. 

Amino acid sequences were obtained from RCSB Protein Data Bank:  
AAV5: 7KP3  
AAV9: 7WJW

#### Merging Sequences  
Merged AAV5 and AAV9 .fasta sequences using Biopython

In [144]:
from Bio import SeqIO
import requests
from io import StringIO

#URLs
AAV5seq = 'https://raw.githubusercontent.com/greercole/AAV_Surface_Aromatics/refs/heads/main/rcsb_pdb_7KP3.fasta'
AAV9seq = 'https://raw.githubusercontent.com/greercole/AAV_Surface_Aromatics/refs/heads/main/rcsb_pdb_7WJW.fasta'

response5 = requests.get(AAV5seq)
response9 = requests.get(AAV9seq)
fasta_data5 = StringIO(response5.text)
fasta_data9 = StringIO(response9.text)


# Load the first FASTA file
seq1 = SeqIO.read(fasta_data5, 'fasta')

# Load the second FASTA file
seq2 = SeqIO.read(fasta_data9, 'fasta')

# Create a list of both sequences
records = [seq1, seq2]

# Save the merged sequence into a new FASTA file
SeqIO.write(records, 'merged.fasta', 'fasta')


2

#### Alignment

Sequences are aligned using Clustal Omega. A key is created to identify positions of amino acids in original sequences. Necessary to back-track misaligned residues.

In [145]:
import subprocess
import requests
from Bio import AlignIO, SeqIO
import pandas as pd
from io import StringIO

def run_clustal_omega(input_fasta, output_clustal):
    """Run Clustal Omega using subprocess."""
    try:
        subprocess.run(["clustalo", "-i", input_fasta, "-o", output_clustal, "--outfmt", "clustal", "--force"], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running Clustal Omega: {e}")
        exit(1)

def read_alignment(output_fasta):
    """Read the Clustal Omega alignment result."""
    try:
        return AlignIO.read(output_fasta, "clustal")
    except FileNotFoundError:
        print("Alignment file not found. Clustal Omega may have failed.")
        return None

def track_positions(original_seq, aligned_seq, original_seq_id):
    """Track residue positions from original to aligned sequence."""
    mapping = []
    original_pos = 0
    aligned_pos = 0
    
    for res_aligned in aligned_seq:
        if res_aligned != "-":  # Ignore gaps in the aligned sequence
            if original_pos < len(original_seq):
                mapping.append({
                    "Sequence_ID": original_seq_id,
                    "Original_Sequence_Position": original_pos + 1,  # 1-based indexing
                    "Aligned_Sequence_Position": aligned_pos + 1,
                    "Residue": res_aligned
                })
                original_pos += 1
        aligned_pos += 1  # Always increment aligned position
    
    return mapping

# **Step 1: Download the merged FASTA file**
merged_fasta_url = "https://raw.githubusercontent.com/greercole/AAV_Surface_Aromatics/main/merged.fasta"
merged_fasta_local = "merged.fasta"
aligned_clustal = "Aligned.clustal"
output_csv = "residue_positions.csv"

response = requests.get(merged_fasta_url)

if response.status_code == 200:
    with open(merged_fasta_local, "w") as f:
        f.write(response.text)
else:
    print("Error: Unable to download the merged FASTA file.")
    exit(1)

# **Step 2: Run Clustal Omega**
run_clustal_omega(merged_fasta_local, aligned_clustal)

# **Step 3: Read original sequences before alignment**
original_sequences = {seq.id: str(seq.seq) for seq in SeqIO.parse(merged_fasta_local, "fasta")}

# **Step 4: Read the alignment**
alignment = read_alignment(aligned_clustal)

# **Step 5: Process aligned sequences**
residue_positions_list = []
if alignment:
    for seq_record in alignment:
        seq_id = seq_record.id
        aligned_seq = str(seq_record.seq)
        original_seq = original_sequences.get(seq_id, "")  # Get unaligned sequence

        # Track positions
        residue_positions = track_positions(original_seq, aligned_seq, seq_id)
        residue_positions_list.extend(residue_positions)

# **Step 6: Convert to DataFrame and save as CSV**
df_residue_positions = pd.DataFrame(residue_positions_list)
df_residue_positions.to_csv(output_csv, index=False)



[Alignment Clustal](/Users/greer/Desktop/AAV5_9_AromaticsAnalysis/mybook/Aligned.clustal)


#### Filtering Mismatches for Aromatic Residues

Identifying all mismatches in alignment that correspond to a Phenylalanine, Tyrosine, Tryptophan, or a Histidine. Create a CSV file of this data.

In [146]:
import csv
from Bio import AlignIO
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from io import BytesIO
import base64
from IPython.display import HTML

# Load alignment
aligned_clustal = "/Users/greer/Desktop/AAV5_9_AromaticsAnalysis/mybook/Aligned.clustal"
alignment = AlignIO.read(aligned_clustal, "clustal")

# Define target residues (aromatic amino acids)
target_residues = {"Y", "W", "H", "F"}

# Convert alignment to a list of sequences
seqs = [str(record.seq) for record in alignment]
seq_ids = [record.id for record in alignment]

# Find mismatches in target residues
mismatch_list = []

# Iterate through positions in the alignment
for i in range(len(seqs[0])):  # Iterate through aligned positions
    column = [seq[i] for seq in seqs]  # Get residues at this aligned position
    unique_residues = set(column)

    # If multiple unique residues and any are in target_residues, it's a mismatch
    if len(unique_residues) > 1 and any(res in target_residues for res in unique_residues):
        for j, res in enumerate(column):
            if res in target_residues:
                # Only record the aligned position and the residue
                mismatch_list.append([seq_ids[j], i + 1, res])  # Aligned position (1-based)

# Define CSV file path
csv_file = "/Users/greer/Desktop/AAV5_9_AromaticsAnalysis/mybook/aromatic_mismatches.csv"

# Save to CSV
with open(csv_file, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Sequence_ID", "Aligned_Sequence_Position", "Residue"])  # Header row
    writer.writerows(mismatch_list)

# Read and sort the mismatches DataFrame
mismatches = pd.read_csv(csv_file)
mismatches = mismatches.sort_values(by='Aligned_Sequence_Position', ascending=True)

mismatches

,Sequence_ID,Aligned_Sequence_Position,Residue
0,7KP3_1|Chain,2,F
1,7KP3_1|Chain,5,H
2,7KP3_1|Chain,9,W
3,7KP3_1|Chain,20,F
4,7KP3_1|Chain,36,H
5,7KP3_1|Chain,48,Y
6,7KP3_1|Chain,50,Y
7,7KP3_1|Chain,73,H
8,7KP3_1|Chain,77,Y
9,7KP3_1|Chain,88,Y


#### Finding Original Sequence Position of Mismatches
Take mismatches and find their locations in original sequences using the key generated above. Again outputing a CSV of this data.

In [147]:
import pandas as pd

# Load the two CSV files
aromatic_mismatches = "/Users/greer/Desktop/AAV5_9_AromaticsAnalysis/mybook/aromatic_mismatches.csv"  # Replace with actual path to the first file
residue_positions = "/Users/greer/Desktop/AAV5_9_AromaticsAnalysis/mybook/residue_positions.csv"  # Replace with actual path to the second file

df1 = pd.read_csv(aromatic_mismatches)  # The file with "Sequence_ID", "Aligned_Position", "Residue"
df2 = pd.read_csv(residue_positions)  # The file with "Original_Sequence_ID", "Original_Sequence_Position", "Aligned_Sequence_Position", "Residue"

# Merge the dataframes on both "Aligned_Sequence_Position" and "Sequence_ID", keeping all columns from df1
merged_df = pd.merge(df1, df2[["Sequence_ID", "Original_Sequence_Position", "Aligned_Sequence_Position", "Residue"]], 
                     on=["Aligned_Sequence_Position", "Sequence_ID"], how="left")

merged_df['Sequence_ID'] = merged_df['Sequence_ID'].str[:4]

# Save the merged result to a new CSV
mismatch_with_original_positon = "/Users/greer/Desktop/AAV5_9_AromaticsAnalysis/mybook/mismatch_with_original_positions.csv"  # Replace with desired output path
merged_df.to_csv(mismatch_with_original_positon, index=False)

merged_df


,Sequence_ID,Aligned_Sequence_Position,Residue_x,Original_Sequence_Position,Residue_y
0,7KP3,2,F,2,F
1,7KP3,5,H,5,H
2,7KP3,9,W,9,W
3,7KP3,20,F,20,F
4,7KP3,36,H,36,H
5,7KP3,48,Y,48,Y
6,7KP3,50,Y,50,Y
7,7KP3,73,H,73,H
8,7KP3,77,Y,77,Y
9,7KP3,88,Y,88,Y


#### Calculating the Solvent Accessible Surface Area of Each Residue
Run Biopython SASA algorithm to determine likelihood of residue to be on the surface of capsid. This uses "Rolling ball" algorithm which uses a sphere (of equal radius to a solvent molecule) to probe the surface of the capsids. Output another CSV.

In [150]:
import pandas as pd
from Bio import PDB

# Enable debug messages
DEBUG = False  

# Paths to input files
pdb_file_1 = "/Users/greer/Desktop/AAV5_9_AromaticsAnalysis/mybook/7kp3.pdb"
cif_file_2 = "/Users/greer/Desktop/AAV5_9_AromaticsAnalysis/mybook/7wjw.cif"

# Load structures
pdb_parser = PDB.PDBParser(QUIET=True)
structure_1 = pdb_parser.get_structure("7KP3", pdb_file_1)

cif_parser = PDB.MMCIFParser()
structure_2 = cif_parser.get_structure("7WJW", cif_file_2)

# Initialize SASA calculator
sr = PDB.SASA.ShrakeRupley()

# Function to calculate per-residue SASA
def calculate_per_residue_sasa(structure, structure_name):
    sasa_data = []
    
    # Calculate SASA for the whole structure
    sr.compute(structure, level="R")  # "R" computes at the residue level

    for model in structure:
        for chain in model:
            for residue in chain:
                residue_name = residue.get_resname().strip()
                residue_num = residue.get_id()[1]  # Residue number
                
                # Extract per-residue SASA
                sasa_value = residue.sasa if hasattr(residue, "sasa") else 0.0

                if DEBUG:
                    print(f"{structure_name} | {residue_name} {residue_num} -> SASA: {sasa_value:.2f}")

                # Adjust residue number based on structure to match original fastas
                if structure_name == "7KP3":
                    residue_num -= 1
                elif structure_name == "7WJW":
                    residue_num -= 218

                sasa_data.append([structure_name, residue_name, residue_num, sasa_value])

    return sasa_data

# Compute per-residue SASA for both structures
sasa_1 = calculate_per_residue_sasa(structure_1, "7KP3")
sasa_2 = calculate_per_residue_sasa(structure_2, "7WJW")

# Store results in a DataFrame
sasa_df = pd.DataFrame(sasa_1 + sasa_2, columns=["Structure", "Residue", "Residue_Position", "SASA"])

# Specify the file path where you want to save the CSV
output_csv_file = "/Users/greer/Desktop/AAV5_9_AromaticsAnalysis/mybook/sasa_output.csv"

# Save the DataFrame to CSV
sasa_df.to_csv(output_csv_file, index=False)


#### Get SASA Values for Mismatched Amino Acids
Finally, line up SASA values to mismatched aromatic residues for both sequences. Clean up output.

In [149]:
import pandas as pd

# Load the first CSV file
df1 = pd.read_csv('/Users/greer/Desktop/AAV5_9_AromaticsAnalysis/mybook/mismatch_with_original_positions.csv')

# Load the second CSV file
df2 = pd.read_csv('/Users/greer/Desktop/AAV5_9_AromaticsAnalysis/mybook/sasa_output.csv')

# Merge the two dataframes based on the matching columns
merged_df = pd.merge(df1, df2, left_on=['Sequence_ID', 'Original_Sequence_Position'], 
                     right_on=['Structure', 'Residue_Position'], how='left')
# Drop the unwanted columns
merged_df = merged_df.drop(columns=['Residue_x', 'Structure', 'Residue', 'Residue_Position'])

# Rename the "Residue_y" column to "Residue"
merged_df = merged_df.rename(columns={'Residue_y': 'Residue'})

# Drop rows where SASA is NaN
merged_df = merged_df.dropna(subset=['SASA'])

# Sort by Sequence_ID and Original_Sequence_Position
merged_df = merged_df.sort_values(by=['Sequence_ID', 'SASA'])

output_csv_file = "/Users/greer/Desktop/AAV5_9_AromaticsAnalysis/mybook/sasa_for_mismatch.csv"

# Save the merged dataframe to a new CSV
merged_df.to_csv(output_csv_file, index=False)

merged_df



,Sequence_ID,Aligned_Sequence_Position,Original_Sequence_Position,Residue,SASA
18,7KP3,282,281,W,5.629634
20,7KP3,294,293,Y,16.344224
25,7KP3,417,416,F,35.021218
48,7KP3,568,562,Y,37.673024
45,7KP3,526,520,Y,38.045743
52,7KP3,590,584,Y,42.912044
31,7KP3,462,456,Y,44.277971
37,7KP3,480,474,W,53.430951
23,7KP3,386,385,F,61.589039
28,7KP3,424,423,F,77.015867


# Conclusions
By running this analysis on a few different serotypes, we can start to identify key aromatics with potential causality for changes in absorbance measurements. This could provide a model to predict absorbance/capsid ratios and eliminate the need to perform physical experiments that require lots of hands on time and material. It also strengthens our titering model and provides explanation for differences in absorbance across serotypes.


----add some screenshots of these residues in model----